In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
!pip install rdkit-pypi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 54.6 MB/s eta 0:00:00


In [ ]:
!pip install py3Dmol

In [12]:
df_final_subcount = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/PSD/tubes/final_subcount.csv')

In [14]:
for i in range(2):
    globals()[f'df_final_subcount{i+1}'] = df_final_subcount[df_final_subcount['cluster_dbscan'] == i]
    globals()[f'df_final_subcount{i+1}'] = globals()[f'df_final_subcount{i+1}'].drop(columns=['cluster_dbscan'])

    globals()[f'df_final_subcount{i+1}'] = globals()[f'df_final_subcount{i+1}'].loc[:, (globals()[f'df_final_subcount{i+1}'] != 0).any(axis=0)]
    globals()[f'df_final_subcount{i+1}'] = globals()[f'df_final_subcount{i+1}'].reset_index(drop=True)

    globals()[f'df_final_subcount{i+1}'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1055 entries, 0 to 1054
Data columns (total 89 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   molecule_chembl_id  1055 non-null   object 
 1   SubFPC1             1055 non-null   float64
 2   SubFPC2             1055 non-null   float64
 3   SubFPC3             1055 non-null   float64
 4   SubFPC4             1055 non-null   float64
 5   SubFPC5             1055 non-null   float64
 6   SubFPC6             1055 non-null   float64
 7   SubFPC8             1055 non-null   float64
 8   SubFPC9             1055 non-null   float64
 9   SubFPC12            1055 non-null   float64
 10  SubFPC13            1055 non-null   float64
 11  SubFPC14            1055 non-null   float64
 12  SubFPC15            1055 non-null   float64
 13  SubFPC16            1055 non-null   float64
 14  SubFPC17            1055 non-null   float64
 15  SubFPC18            1055 non-null   float64
 16  SubFPC

In [15]:
import pandas as pd
for i in range(2):
  globals()[f'data{i+1}'] = globals()[f'df_final_subcount{i+1}'].drop(columns=['molecule_chembl_id'])

In [16]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

def kennard_stone_sampling(data, num_samples):
    scaler = MinMaxScaler()
    data = scaler.fit_transform(data)

    num_data = data.shape[0]
    distance_matrix = np.sqrt(((data[:, None, :] - data[None, :, :]) ** 2).sum(axis=2))

    i, j = np.unravel_index(np.argmax(distance_matrix), distance_matrix.shape)
    selected_indices = [i, j]
    remaining_indices = list(set(range(num_data)) - set(selected_indices))

    while len(selected_indices) < num_samples:
        min_distances = np.min(distance_matrix[remaining_indices][:, selected_indices], axis=1)
        next_index = remaining_indices[np.argmax(min_distances)]
        selected_indices.append(next_index)
        remaining_indices.remove(next_index)

    return selected_indices

In [17]:
num_samples = [20, 10]
selected_data = {}

for i in range(2):
    selected_indices = kennard_stone_sampling(globals()[f'data{i+1}'], num_samples[i])
    selected_data[i+1] = globals()[f'df_final_subcount{i+1}'].iloc[selected_indices]
    print(f"Data sampel terpilih di cluster {i}:")
    print(selected_data[i+1])

Data sampel terpilih di cluster 0:
     molecule_chembl_id  SubFPC1  SubFPC2  SubFPC3  SubFPC4  SubFPC5  SubFPC6  \
424       CHEMBL1097763      1.0      0.0      0.0      0.0      0.0      0.0   
743       CHEMBL4563511      0.0      0.0      0.0      0.0      0.0      0.0   
656       CHEMBL3622075      0.0      1.0      0.0      1.0      0.0      0.0   
798       CHEMBL5283822      1.0      0.0      0.0      0.0      0.0      1.0   
405       CHEMBL1095413      0.0      0.0      0.0      0.0      1.0      0.0   
1047       CHEMBL348263      1.0      2.0      0.0      0.0      0.0      0.0   
475       CHEMBL1688909      0.0      0.0      0.0      0.0      0.0      0.0   
777           CHEMBL185      1.0      0.0      0.0      0.0      0.0      0.0   
734       CHEMBL4444996      0.0      0.0      0.0      0.0      0.0      0.0   
320        CHEMBL496375      0.0      0.0      0.0      0.0      0.0      0.0   
879       CHEMBL5421289      4.0      3.0      2.0      2.0      0.0      

In [18]:
df_final_active_potential_compound = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/PSD/tubes/final_active_potential_compound_with_cluster.csv')

In [19]:
for i in range(2):
  globals()[f'sample_molecule{i+1}'] = df_final_active_potential_compound[df_final_active_potential_compound['molecule_chembl_id'].isin(selected_data[i+1]['molecule_chembl_id'])]

In [20]:
sample_molecule_chembl_id = {}
for i in range(2):
    sample_molecule_chembl_id[i+1] = globals()[f'sample_molecule{i+1}']['molecule_chembl_id'].tolist()
    print(sample_molecule_chembl_id[i+1])

['CHEMBL355905', 'CHEMBL300109', 'CHEMBL463453', 'CHEMBL496375', 'CHEMBL1095413', 'CHEMBL1097763', 'CHEMBL369471', 'CHEMBL1688909', 'CHEMBL1784791', 'CHEMBL2312962', 'CHEMBL2442756', 'CHEMBL3622075', 'CHEMBL4228244', 'CHEMBL4444996', 'CHEMBL4563511', 'CHEMBL185', 'CHEMBL5283822', 'CHEMBL5421289', 'CHEMBL5402619', 'CHEMBL348263']
['CHEMBL27879', 'CHEMBL165106', 'CHEMBL74295', 'CHEMBL129883', 'CHEMBL1097371', 'CHEMBL3407536', 'CHEMBL3623215', 'CHEMBL3753842', 'CHEMBL3245357', 'CHEMBL3752192']


In [21]:
sample_molecule_smiles = {}
for i in range(2):
    sample_molecule_smiles[i+1] = globals()[f'sample_molecule{i+1}']['canonical_smiles'].tolist()
    print(sample_molecule_smiles[i+1])

['CSc1nc2ccccn2c(=N)c1S(=O)(=O)c1ccccc1', 'c1cnc2c(c1)C(Cn1ccnc1)CCC2', 'O=C1c2c(O)cc(O)cc2O[C@H](c2ccc(O)cc2O)[C@H]1O', 'COc1ccc(-c2cc3c(=O)cc(O)oc3cc2SCc2ccncc2)cc1', 'O=C1/C(=C\\c2cccnc2)C(c2ccc(OC(F)(F)F)cc2)n2cccc21', 'Cc1cccc(O)c1C(=O)c1c(O)cc(C(=O)O)cc1C=O', 'O=C1OC[C@H](Cc2ccc(O)c(O)c2)[C@H]1Cc1ccc(O)c(O)c1', 'CN(C)c1ccc(Oc2ccc3c(Cn4ccnc4)cc(=O)oc3c2)cc1', 'COc1cc(CC(=O)N2CCOC(Cn3ccnc3)C2)cc(OC)c1OC', 'Cc1cccc(C(c2ccncc2)c2cc3c4c(c2)CCN4C(=O)CC3)c1', 'c1ccc(-c2cc(Cn3ccnc3)c3ccc4[nH]ccc4c3n2)cc1', 'N#CC1(c2cc(Cn3cncn3)cc(Sc3ccc(OS(N)(=O)=O)cc3)c2)CC1', 'c1cnc(-c2nc(-c3nc4ccccc4s3)cs2)nc1', 'N#Cc1ccc(S(=O)(=O)NCc2cccc(CNS(=O)(=O)c3ccc(C#N)cc3)c2)cc1', 'O=[N+]([O-])c1cccc(CN(c2ccc([N+](=O)[O-])c(-c3ccccc3)c2)n2cnnc2)c1', 'O=c1[nH]cc(F)c(=O)[nH]1', 'CC#CCOc1ccc2cc(C(O)(c3ccc(Br)cc3)c3cccnc3)oc2c1', 'C[C@@H]1[C@@H](OC(=O)Cc2ccncc2)C[C@H]2C[C@]1(C)C2(C)C', 'N#Cc1ccc(N(CCSCc2ccc(O)cc2)n2cnnc2)cc1', 'CCCCOc1ccc(N/C=N\\O)cc1']
['CCCCCCCCCCN1C(=O)C2CC(c3ccc(N)cc3)(C2)C1=O', 'C[C@]12CC=C3

In [22]:
from rdkit import Chem
from rdkit.Chem import AllChem

for i in range(len(sample_molecule_smiles)):
    smiles_list = sample_molecule_smiles[i + 1]
    for j in range(len(smiles_list)):
        smiles = smiles_list[j]
        mol = Chem.MolFromSmiles(smiles)
        mol = Chem.AddHs(mol)

        result = AllChem.EmbedMolecule(mol)
        if result == 0:
            AllChem.UFFOptimizeMolecule(mol)

            chembl_id = sample_molecule_chembl_id[i + 1]
            if isinstance(chembl_id, list):
                chembl_id = chembl_id[j]

            pdb_filename = f"{chembl_id}.pdb"
            Chem.MolToPDBFile(mol, pdb_filename)

            print(f"File PDB untuk ChemBL ID {chembl_id} telah disimpan sebagai {pdb_filename}.")
        else:
            print(f"Embedding gagal untuk SMILES {smiles}. Molekul {chembl_id} tidak diproses.")

File PDB untuk ChemBL ID CHEMBL355905 telah disimpan sebagai CHEMBL355905.pdb.
File PDB untuk ChemBL ID CHEMBL300109 telah disimpan sebagai CHEMBL300109.pdb.
File PDB untuk ChemBL ID CHEMBL463453 telah disimpan sebagai CHEMBL463453.pdb.
File PDB untuk ChemBL ID CHEMBL496375 telah disimpan sebagai CHEMBL496375.pdb.
File PDB untuk ChemBL ID CHEMBL1095413 telah disimpan sebagai CHEMBL1095413.pdb.
File PDB untuk ChemBL ID CHEMBL1097763 telah disimpan sebagai CHEMBL1097763.pdb.
File PDB untuk ChemBL ID CHEMBL369471 telah disimpan sebagai CHEMBL369471.pdb.
File PDB untuk ChemBL ID CHEMBL1688909 telah disimpan sebagai CHEMBL1688909.pdb.
File PDB untuk ChemBL ID CHEMBL1784791 telah disimpan sebagai CHEMBL1784791.pdb.
File PDB untuk ChemBL ID CHEMBL2312962 telah disimpan sebagai CHEMBL2312962.pdb.
File PDB untuk ChemBL ID CHEMBL2442756 telah disimpan sebagai CHEMBL2442756.pdb.
File PDB untuk ChemBL ID CHEMBL3622075 telah disimpan sebagai CHEMBL3622075.pdb.
File PDB untuk ChemBL ID CHEMBL4228244

In [24]:
import py3Dmol
import os

for i in range(2):
    print(f'\033[1mCluster {i}\033[0m')
    for j in range(len(sample_molecule_chembl_id[i+1])):
        chembl_id = sample_molecule_chembl_id[i+1][j]

        pdb_filename = f'{chembl_id}.pdb'

        if not os.path.exists(pdb_filename):
            print(f'Skipping {chembl_id}: File not found.')
            continue

        with open(pdb_filename, 'r') as pdb_file:
            pdb_data = pdb_file.read()

            view = py3Dmol.view(width=800, height=400)
            view.addModel(pdb_data, "pdb")
            view.setStyle({'stick': {}})
            view.zoomTo()
            view.show()

Cluster 0


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

...

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [ ]:
import py3Dmol

protein_directory = "/content/drive/MyDrive/Colab Notebooks/PSD/tubes"
pdb_id = "4kq8"

protein_file = f"{protein_directory}/{pdb_id}.pdb"

with open(protein_file, 'r') as pdb_file:
    pdb_data = pdb_file.read()

view = py3Dmol.view(width=800, height=800)
view.addModel(pdb_data, "pdb")
view.setStyle({'stick': {}})
view.zoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [ ]:
import MDAnalysis as mda

u = mda.Universe(protein_file)

protein = u.select_atoms("protein")
ligand = u.select_atoms("resname LET")  # LET : Letrazole, ANA/ANS : Anastrazole, EXE : Exemestan
water = u.select_atoms("resname HOH")  # Molekul air (resname HOH)

print(f"Protein: {protein.n_atoms} atom")
print(f"Ligand: {ligand.n_atoms} atom")
print(f"Water: {water.n_atoms} atom")

Protein: 3658 atom
Ligand: 0 atom
Water: 2 atom
